In [3]:
#@title　基本 import
import sys
import json
import torch
import textwrap
import torchsnooper
import pytorch_transformers
import torch.nn.functional as F
from tqdm import trange
from IPython.core.display import display, HTML

In [4]:
#@title 下載 GPT-Chinese

GITHUB_REPO = "GPT2-Chinese"
!rm -rf {GITHUB_REPO}
!git clone https://github.com/Morizeyao/{GITHUB_REPO}.git {GITHUB_REPO}
if not GITHUB_REPO in sys.path:
    sys.path += [GITHUB_REPO]

Cloning into 'GPT2-Chinese'...
remote: Enumerating objects: 199, done.
remote: Total 199 (delta 0), reused 0 (delta 0), pack-reused 199
Receiving objects: 100% (199/199), 13.41 MiB | 1.49 MiB/s, done.
Resolving deltas: 100% (97/97), done.


In [5]:
#@title 下載、解壓縮模型
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Z8WdVYgBj01BHU4syjlY9qj3KBfEFP2D' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1Z8WdVYgBj01BHU4syjlY9qj3KBfEFP2D" -O 10layers_12heads_1024len_768embd_full_corpus_16bsize.zip && rm -rf /tmp/cookies.txt

pretrained_model = '10layers_12heads_1024len_768embd_full_corpus_16bsize'

!unzip {pretrained_model}.zip
sagemaker_base_path = 'home/ec2-user/SageMaker/tmp/GPT2-Chinese'
config_file = 'config.json'
model_ckpt = "pytorch_model.bin"
vocab_file = "vocab_small.txt"

!rm {config_file} {model_ckpt} {vocab_file}

!mv {sagemaker_base_path}/model/{pretrained_model}/{config_file} {config_file}
!mv {sagemaker_base_path}/model/{pretrained_model}/{model_ckpt} {model_ckpt}
!mv {sagemaker_base_path}/cache/{vocab_file} {vocab_file}

sed: illegal option -- r
usage: sed script [-Ealn] [-i extension] [file ...]
       sed [-Ealn] [-i extension] [-e script] ... [-f script_file] ... [file ...]
--2020-02-03 11:35:34--  https://docs.google.com/uc?export=download&confirm=&id=1Z8WdVYgBj01BHU4syjlY9qj3KBfEFP2D
Resolving docs.google.com (docs.google.com)... 172.217.160.78
Connecting to docs.google.com (docs.google.com)|172.217.160.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘10layers_12heads_1024len_768embd_full_corpus_16bsize.zip’

10layers_12heads_10     [ <=>                ]   3.24K  --.-KB/s    in 0s      

2020-02-03 11:35:35 (6.46 MB/s) - ‘10layers_12heads_1024len_768embd_full_corpus_16bsize.zip’ saved [3318]

Archive:  10layers_12heads_1024len_768embd_full_corpus_16bsize.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central direct

In [6]:
#@title 載入模型以及 Tokenizer

from tokenizations import tokenization_bert

# make model output attentions
config = pytorch_transformers.GPT2Config.from_json_file(config_file)
config.output_attentions = True


model = pytorch_transformers.GPT2LMHeadModel.from_pretrained(".", config=config)
tokenizer = tokenization_bert.BertTokenizer(vocab_file=vocab_file)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()
clear_output()

FileNotFoundError: [Errno 2] No such file or directory: 'config.json'